In [1]:
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
def special_score(preds, label):

    fpr, tpr, thresholds = roc_curve(label,preds,pos_label=1)
    
    ind = (pd.Series(fpr)-0.001).abs().sort_values().index[0]
    tpr1 = pd.Series(tpr).loc[ind]
    
    ind = (pd.Series(fpr)-0.005).abs().sort_values().index[0]
    tpr2 = pd.Series(tpr).loc[ind]
    
    ind = (pd.Series(fpr)-0.01).abs().sort_values().index[0]
    tpr3 = pd.Series(tpr).loc[ind]
    
    return tpr1*0.4+tpr2*0.3+tpr3*0.3
kf = KFold(n_splits=5, shuffle = True, random_state=2018)
params_initial = {
        'num_leaves':95, 
        'learning_rate':0.16, 
    'boosting':'dart',
    'min_child_samples':10,
    'subsample':1.0, 
    'colsample_bytree':0.8, 
    'reg_alpha':1,
    'reg_lambda':2, 
    'seed':2018,
    'metric':'auc',
    'objective':'binary'
}
train_x_sparse=sparse.load_npz('preprocess/train_x_sparse.npz')
test_x_sparse=sparse.load_npz('preprocess/test_x_sparse.npz')
res = pd.read_csv('preprocess/res_id_test.csv')
train_y = pd.read_csv('preprocess/train_y.csv',header=None)[0]
print(train_x_sparse.shape)
print (test_x_sparse.shape)

def five_flod_score(params):
    score = []
    for train_index,evals_index in kf.split(train_y):
        train_part = lgb.Dataset(train_x_sparse[train_index.tolist(),:],label=train_y.loc[train_index.tolist()])
        evals = lgb.Dataset(train_x_sparse[evals_index.tolist(),:],label=train_y.loc[evals_index.tolist()])
        model = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,
              init_model=None, feature_name='auto', 
              categorical_feature='auto', early_stopping_rounds=50,
              evals_result=None, verbose_eval=True, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
        ypre = model.predict(train_x_sparse[evals_index.tolist(),:])
        s = special_score(ypre,train_y.loc[evals_index.tolist()])
        score.append(s)
        print('比赛参考得分为',s)
        print('\n')
        print('\n')
    print(score)
    mean_score = np.array(score).mean()
    print('比赛平均参考得分为',mean_score)
    print('\n')
    print('\n')
    return mean_score

def get_best_params(params):
    params_choose = {}
    params_choose['num_leaves'] = list(range(20,120,10))
    params_choose['min_child_samples'] = list(range(5,50,5))
    params_choose['boosting'] = ['gbdt','dart']
    params_choose['subsample'] = list(np.array(list(range(5,11)))*0.1)
    params_choose['colsample_bytree'] = list(np.array(list(range(5,11)))*0.1)
    params_choose['reg_alpha'] = list(range(5))
    params_choose['reg_lambda'] = list(range(5))
    params_choose['learning_rate'] = list(np.array(list(range(1,11)))*0.02)
    params_choose['metric'] = ['auc','binary_logloss']
    for key in params_choose.keys():
        value = params_choose[key]
        score = pd.Series()
        for va in value:
            params[key] = va
            score = score.append(pd.Series(five_flod_score(params),index=[va]))
            print(key,va)
            print('\n')
        score = score.sort_values()
        best_score = score.values[-1]
        params[key] = score.index[-1]
        print(key,best_score)
        pd.Series(params).to_csv('preprocess/best_params_lgb.csv')
        print('\n')
    return params

import os
if os.path.exists('preprocess/best_params_lgb.csv'):
    best_params = pd.read_csv('preprocess/best_params_lgb.csv',header=None).set_index(0).to_dict()[1]
else:
    best_params = get_best_params(params_initial)
    pd.Series(best_params).to_csv('preprocess/best_params_lgb.csv')
def get_predict_result(params,test_x_sparse):
    num = 0
    score = []
    for train_index,evals_index in kf.split(train_y):
        train_part = lgb.Dataset(train_x_sparse[train_index.tolist(),:],label=train_y.loc[train_index.tolist()])
        evals = lgb.Dataset(train_x_sparse[evals_index.tolist(),:],label=train_y.loc[evals_index.tolist()])
        bst = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,feval=None,feature_name='auto', 
              categorical_feature='auto', early_stopping_rounds=50,
              evals_result=None, verbose_eval=True, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
        num+=1
        evals_ypre = bst.predict(train_x_sparse[evals_index.tolist(),:])
        score.append(special_score(evals_ypre,train_y.loc[evals_index.tolist()]))
        test_ypre = pd.Series(bst.predict(test_x_sparse))
        res['lgb'+str(num)] = test_ypre
    return res,score

result,score = get_predict_result(best_params,test_x_sparse)
result['lgb'] = (result['lgb1']+result['lgb2']+result['lgb3']+result['lgb4']+result['lgb5'])/5
result.to_csv('result/five_fold_all.csv',index=False)
print(result.describe())
col = ['lgb','lgb1', 'lgb2', 'lgb3', 'lgb4', 'lgb5']
for co in col:
    sub = result[['id',co]]
    sub.columns = ['id','score']
    sub.to_csv('result/five_fold_'+co+'.csv',index=False)
    print(co)
print('Over')

(990006, 3554)
(500538, 3554)
[1]	train's auc: 0.914519	evals's auc: 0.91558
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.945474	evals's auc: 0.945127
[3]	train's auc: 0.961534	evals's auc: 0.962673
[4]	train's auc: 0.964815	evals's auc: 0.965812
[5]	train's auc: 0.968974	evals's auc: 0.968135
[6]	train's auc: 0.972202	evals's auc: 0.970546
[7]	train's auc: 0.973746	evals's auc: 0.972273
[8]	train's auc: 0.974826	evals's auc: 0.972882
[9]	train's auc: 0.976099	evals's auc: 0.973961
[10]	train's auc: 0.976606	evals's auc: 0.97458
[11]	train's auc: 0.977162	evals's auc: 0.974807
[12]	train's auc: 0.97837	evals's auc: 0.975474
[13]	train's auc: 0.979733	evals's auc: 0.97657
[14]	train's auc: 0.980123	evals's auc: 0.97686
[15]	train's auc: 0.980233	evals's auc: 0.977218
[16]	train's auc: 0.98119	evals's auc: 0.978065
[17]	train's auc: 0.981761	evals's auc: 0.978788
[18]	train's auc: 0.982076	evals's auc: 0.979292
[19]	train's auc: 0.982443	evals's auc: 0

[167]	train's auc: 0.991648	evals's auc: 0.988297
[168]	train's auc: 0.991692	evals's auc: 0.988327
[169]	train's auc: 0.991734	evals's auc: 0.988376
[170]	train's auc: 0.991728	evals's auc: 0.988369
[171]	train's auc: 0.991761	evals's auc: 0.98835
[172]	train's auc: 0.991753	evals's auc: 0.988354
[173]	train's auc: 0.991731	evals's auc: 0.988344
[174]	train's auc: 0.991775	evals's auc: 0.988377
[175]	train's auc: 0.991792	evals's auc: 0.988382
[176]	train's auc: 0.991776	evals's auc: 0.988366
[177]	train's auc: 0.991774	evals's auc: 0.98837
[178]	train's auc: 0.991786	evals's auc: 0.988398
[179]	train's auc: 0.991864	evals's auc: 0.988475
[180]	train's auc: 0.991896	evals's auc: 0.988496
[181]	train's auc: 0.991911	evals's auc: 0.988549
[182]	train's auc: 0.991896	evals's auc: 0.988538
[183]	train's auc: 0.991886	evals's auc: 0.988531
[184]	train's auc: 0.991876	evals's auc: 0.988525
[185]	train's auc: 0.991909	evals's auc: 0.988559
[186]	train's auc: 0.991899	evals's auc: 0.988554
[1

[332]	train's auc: 0.99314	evals's auc: 0.989403
[333]	train's auc: 0.993153	evals's auc: 0.989424
[334]	train's auc: 0.99316	evals's auc: 0.989384
[335]	train's auc: 0.993181	evals's auc: 0.98939
[336]	train's auc: 0.993217	evals's auc: 0.98941
[337]	train's auc: 0.993208	evals's auc: 0.989405
[338]	train's auc: 0.993201	evals's auc: 0.989398
[339]	train's auc: 0.993196	evals's auc: 0.989394
[340]	train's auc: 0.993224	evals's auc: 0.989363
[341]	train's auc: 0.993216	evals's auc: 0.989358
[342]	train's auc: 0.993237	evals's auc: 0.989374
[343]	train's auc: 0.99325	evals's auc: 0.989384
[344]	train's auc: 0.993279	evals's auc: 0.989445
[345]	train's auc: 0.993306	evals's auc: 0.989423
[346]	train's auc: 0.993298	evals's auc: 0.989418
[347]	train's auc: 0.993327	evals's auc: 0.989405
[348]	train's auc: 0.993339	evals's auc: 0.98942
[349]	train's auc: 0.993357	evals's auc: 0.989417
[350]	train's auc: 0.993378	evals's auc: 0.989424
[351]	train's auc: 0.993372	evals's auc: 0.989421
[352]	

[497]	train's auc: 0.994428	evals's auc: 0.989843
[498]	train's auc: 0.99444	evals's auc: 0.989852
[499]	train's auc: 0.994471	evals's auc: 0.989873
[500]	train's auc: 0.994465	evals's auc: 0.989871
[501]	train's auc: 0.994461	evals's auc: 0.989869
[502]	train's auc: 0.994498	evals's auc: 0.989881
[503]	train's auc: 0.994518	evals's auc: 0.989885
[504]	train's auc: 0.994523	evals's auc: 0.98988
[505]	train's auc: 0.994529	evals's auc: 0.989887
[506]	train's auc: 0.994528	evals's auc: 0.989884
[507]	train's auc: 0.994541	evals's auc: 0.989887
[508]	train's auc: 0.994536	evals's auc: 0.989885
[509]	train's auc: 0.99457	evals's auc: 0.989851
[510]	train's auc: 0.994565	evals's auc: 0.989847
[511]	train's auc: 0.994559	evals's auc: 0.989844
[512]	train's auc: 0.994551	evals's auc: 0.989845
[513]	train's auc: 0.99457	evals's auc: 0.989867
[514]	train's auc: 0.994566	evals's auc: 0.989866
[515]	train's auc: 0.994571	evals's auc: 0.989874
[516]	train's auc: 0.994579	evals's auc: 0.98988
[517]

[5]	train's auc: 0.972934	evals's auc: 0.97144
[6]	train's auc: 0.973464	evals's auc: 0.972322
[7]	train's auc: 0.975009	evals's auc: 0.974518
[8]	train's auc: 0.976975	evals's auc: 0.97669
[9]	train's auc: 0.977182	evals's auc: 0.977161
[10]	train's auc: 0.978013	evals's auc: 0.977984
[11]	train's auc: 0.97901	evals's auc: 0.978438
[12]	train's auc: 0.979768	evals's auc: 0.979163
[13]	train's auc: 0.980023	evals's auc: 0.979713
[14]	train's auc: 0.980932	evals's auc: 0.980643
[15]	train's auc: 0.981389	evals's auc: 0.980959
[16]	train's auc: 0.981763	evals's auc: 0.981132
[17]	train's auc: 0.982861	evals's auc: 0.982189
[18]	train's auc: 0.983119	evals's auc: 0.982288
[19]	train's auc: 0.983467	evals's auc: 0.982657
[20]	train's auc: 0.983756	evals's auc: 0.982939
[21]	train's auc: 0.983952	evals's auc: 0.983072
[22]	train's auc: 0.98408	evals's auc: 0.983258
[23]	train's auc: 0.984484	evals's auc: 0.983769
[24]	train's auc: 0.984644	evals's auc: 0.98392
[25]	train's auc: 0.984927	eva

...

[491]	train's auc: 0.995289	evals's auc: 0.991337
[492]	train's auc: 0.995286	evals's auc: 0.991337
[493]	train's auc: 0.995308	evals's auc: 0.99136
[494]	train's auc: 0.995304	evals's auc: 0.991358
[495]	train's auc: 0.995303	evals's auc: 0.991357
[496]	train's auc: 0.9953	evals's auc: 0.991356
[497]	train's auc: 0.995313	evals's auc: 0.991345
[498]	train's auc: 0.995336	evals's auc: 0.991351
[499]	train's auc: 0.995356	evals's auc: 0.991355
[500]	train's auc: 0.995351	evals's auc: 0.991353
[501]	train's auc: 0.995347	evals's auc: 0.99135
[502]	train's auc: 0.995378	evals's auc: 0.991373
[503]	train's auc: 0.9954	evals's auc: 0.991382
[504]	train's auc: 0.995415	evals's auc: 0.991376
[505]	train's auc: 0.995414	evals's auc: 0.991376
[506]	train's auc: 0.995414	evals's auc: 0.991376
[507]	train's auc: 0.995431	evals's auc: 0.991391
[508]	train's auc: 0.995427	evals's auc: 0.991389
[509]	train's auc: 0.995451	evals's auc: 0.991396
[510]	train's auc: 0.995448	evals's auc: 0.991395
[511]	

[656]	train's auc: 0.996356	evals's auc: 0.991751
[657]	train's auc: 0.996356	evals's auc: 0.991751
[658]	train's auc: 0.996354	evals's auc: 0.99175
[659]	train's auc: 0.996363	evals's auc: 0.991757
[660]	train's auc: 0.996366	evals's auc: 0.991759
[661]	train's auc: 0.996363	evals's auc: 0.991758
[662]	train's auc: 0.996364	evals's auc: 0.991758
[663]	train's auc: 0.996369	evals's auc: 0.991755
[664]	train's auc: 0.996378	evals's auc: 0.991755
[665]	train's auc: 0.996375	evals's auc: 0.991754
[666]	train's auc: 0.996374	evals's auc: 0.991754
[667]	train's auc: 0.996372	evals's auc: 0.991753
[668]	train's auc: 0.99637	evals's auc: 0.991752
[669]	train's auc: 0.99637	evals's auc: 0.991751
[670]	train's auc: 0.99637	evals's auc: 0.991751
[671]	train's auc: 0.996377	evals's auc: 0.991756
[672]	train's auc: 0.996377	evals's auc: 0.991755
[673]	train's auc: 0.996381	evals's auc: 0.991751
[674]	train's auc: 0.996383	evals's auc: 0.99175
[675]	train's auc: 0.996393	evals's auc: 0.991747
[676]

[821]	train's auc: 0.9969	evals's auc: 0.991944
[822]	train's auc: 0.996903	evals's auc: 0.991944
[823]	train's auc: 0.996908	evals's auc: 0.991943
[824]	train's auc: 0.99691	evals's auc: 0.991944
[825]	train's auc: 0.996921	evals's auc: 0.991942
[826]	train's auc: 0.99692	evals's auc: 0.991941
[827]	train's auc: 0.99692	evals's auc: 0.991941
[828]	train's auc: 0.996918	evals's auc: 0.99194
[829]	train's auc: 0.996921	evals's auc: 0.991947
[830]	train's auc: 0.996921	evals's auc: 0.991947
[831]	train's auc: 0.996927	evals's auc: 0.991964
[832]	train's auc: 0.996929	evals's auc: 0.991965
[833]	train's auc: 0.996938	evals's auc: 0.991965
[834]	train's auc: 0.996937	evals's auc: 0.991966
[835]	train's auc: 0.996943	evals's auc: 0.991975
[836]	train's auc: 0.996941	evals's auc: 0.991974
[837]	train's auc: 0.996943	evals's auc: 0.991973
[838]	train's auc: 0.996942	evals's auc: 0.991973
[839]	train's auc: 0.996947	evals's auc: 0.991975
[840]	train's auc: 0.996945	evals's auc: 0.991974
[841]	

[986]	train's auc: 0.997388	evals's auc: 0.992227
[987]	train's auc: 0.997387	evals's auc: 0.992227
[988]	train's auc: 0.997391	evals's auc: 0.992232
[989]	train's auc: 0.997389	evals's auc: 0.992232
[990]	train's auc: 0.997388	evals's auc: 0.992231
[991]	train's auc: 0.997399	evals's auc: 0.992232
[992]	train's auc: 0.997398	evals's auc: 0.992231
[993]	train's auc: 0.997399	evals's auc: 0.992231
[994]	train's auc: 0.997398	evals's auc: 0.992231
[995]	train's auc: 0.997403	evals's auc: 0.992227
[996]	train's auc: 0.997408	evals's auc: 0.992233
[997]	train's auc: 0.997419	evals's auc: 0.992234
[998]	train's auc: 0.997427	evals's auc: 0.992237
[999]	train's auc: 0.997427	evals's auc: 0.992236
[1000]	train's auc: 0.997431	evals's auc: 0.992238
[1001]	train's auc: 0.997431	evals's auc: 0.992238
[1002]	train's auc: 0.997431	evals's auc: 0.992239
[1003]	train's auc: 0.997437	evals's auc: 0.992239
[1004]	train's auc: 0.997444	evals's auc: 0.992236
[1005]	train's auc: 0.997442	evals's auc: 0.9

                lgb1           lgb2           lgb3           lgb4  \
count  500538.000000  500538.000000  500538.000000  500538.000000   
mean        0.010182       0.009497       0.011203       0.010792   
std         0.050819       0.051746       0.051765       0.050931   
min         0.000017       0.000003       0.000036       0.000022   
25%         0.000189       0.000111       0.000291       0.000247   
50%         0.000646       0.000414       0.000887       0.000769   
75%         0.003245       0.002385       0.004067       0.003910   
max         0.929255       0.955034       0.931374       0.927114   

                lgb5            lgb  
count  500538.000000  500538.000000  
mean        0.009624       0.010260  
std         0.051520       0.051003  
min         0.000006       0.000020  
25%         0.000124       0.000199  
50%         0.000448       0.000650  
75%         0.002497       0.003293  
max         0.946478       0.933407  
lgb
lgb1
lgb2
lgb3
lgb4
lgb5
Over


In [3]:
print(score)
print(np.array(score).mean())

[0.6655269509251811, 0.6589166666666667, 0.6454659949622167, 0.6724958263772955, 0.6621236777868185]
0.6609058233436358


In [5]:
from sklearn.metrics import roc_curve
import pandas as pd
import numpy as np
from scipy import sparse
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings("ignore")
import lightgbm as lgb
def special_score(preds, label):

    fpr, tpr, thresholds = roc_curve(label,preds,pos_label=1)
    
    ind = (pd.Series(fpr)-0.001).abs().sort_values().index[0]
    tpr1 = pd.Series(tpr).loc[ind]
    
    ind = (pd.Series(fpr)-0.005).abs().sort_values().index[0]
    tpr2 = pd.Series(tpr).loc[ind]
    
    ind = (pd.Series(fpr)-0.01).abs().sort_values().index[0]
    tpr3 = pd.Series(tpr).loc[ind]
    
    return tpr1*0.4+tpr2*0.3+tpr3*0.3
kf = KFold(n_splits=5, shuffle = True, random_state=2018)
params_initial = {
        'num_leaves':95, 
        'learning_rate':0.16, 
    'boosting':'dart',
    'min_child_samples':10,
    'subsample':1.0, 
    'colsample_bytree':0.8, 
    'reg_alpha':1,
    'reg_lambda':2, 
    'seed':2018,
    'metric':'auc',
    'objective':'binary'
}
train_x_sparse=sparse.load_npz('preprocess/train_x_sparse.npz')
date = pd.read_csv('preprocess/train_day.csv')
train_x_sparse = sparse.hstack((train_x_sparse,date)).tocsr()


test_x_sparse=sparse.load_npz('preprocess/test_x_sparse.npz')
date = pd.read_csv('preprocess/test_day.csv')
test_x_sparse = sparse.hstack((test_x_sparse,date)).tocsr()

res = pd.read_csv('preprocess/res_id_test.csv')
train_y = pd.read_csv('preprocess/train_y.csv',header=None)[0]
print(train_x_sparse.shape)
print (test_x_sparse.shape)

def five_flod_score(params):
    score = []
    for train_index,evals_index in kf.split(train_y):
        train_part = lgb.Dataset(train_x_sparse[train_index.tolist(),:],label=train_y.loc[train_index.tolist()])
        evals = lgb.Dataset(train_x_sparse[evals_index.tolist(),:],label=train_y.loc[evals_index.tolist()])
        model = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,
              init_model=None, feature_name='auto', 
              categorical_feature='auto', early_stopping_rounds=50,
              evals_result=None, verbose_eval=True, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
        ypre = model.predict(train_x_sparse[evals_index.tolist(),:])
        s = special_score(ypre,train_y.loc[evals_index.tolist()])
        score.append(s)
        print('比赛参考得分为',s)
        print('\n')
        print('\n')
    print(score)
    mean_score = np.array(score).mean()
    print('比赛平均参考得分为',mean_score)
    print('\n')
    print('\n')
    return mean_score

def get_best_params(params):
    params_choose = {}
    params_choose['num_leaves'] = list(range(20,120,10))
    params_choose['min_child_samples'] = list(range(5,50,5))
    params_choose['boosting'] = ['gbdt','dart']
    params_choose['subsample'] = list(np.array(list(range(5,11)))*0.1)
    params_choose['colsample_bytree'] = list(np.array(list(range(5,11)))*0.1)
    params_choose['reg_alpha'] = list(range(5))
    params_choose['reg_lambda'] = list(range(5))
    params_choose['learning_rate'] = list(np.array(list(range(1,11)))*0.02)
    params_choose['metric'] = ['auc','binary_logloss']
    for key in params_choose.keys():
        value = params_choose[key]
        score = pd.Series()
        for va in value:
            params[key] = va
            score = score.append(pd.Series(five_flod_score(params),index=[va]))
            print(key,va)
            print('\n')
        score = score.sort_values()
        best_score = score.values[-1]
        params[key] = score.index[-1]
        print(key,best_score)
        pd.Series(params).to_csv('preprocess/best_params_lgb.csv')
        print('\n')
    return params

import os
if os.path.exists('preprocess/best_params_lgb.csv'):
    best_params = pd.read_csv('preprocess/best_params_lgb.csv',header=None).set_index(0).to_dict()[1]
else:
    best_params = get_best_params(params_initial)
    pd.Series(best_params).to_csv('preprocess/best_params_lgb.csv')
def get_predict_result(params,test_x_sparse):
    num = 0
    score = []
    for train_index,evals_index in kf.split(train_y):
        train_part = lgb.Dataset(train_x_sparse[train_index.tolist(),:],label=train_y.loc[train_index.tolist()])
        evals = lgb.Dataset(train_x_sparse[evals_index.tolist(),:],label=train_y.loc[evals_index.tolist()])
        bst = lgb.train(params,train_part, 
              num_boost_round=10000, valid_sets=[train_part,evals], 
              valid_names=['train','evals'], fobj=None,feval=None,feature_name='auto', 
              categorical_feature='auto', early_stopping_rounds=50,
              evals_result=None, verbose_eval=True, learning_rates=None, 
              keep_training_booster=False, callbacks=None)
        num+=1
        evals_ypre = bst.predict(train_x_sparse[evals_index.tolist(),:])
        score.append(special_score(evals_ypre,train_y.loc[evals_index.tolist()]))
        test_ypre = pd.Series(bst.predict(test_x_sparse))
        res['lgb'+str(num)] = test_ypre
    return res,score

result,score = get_predict_result(best_params,test_x_sparse)
result['lgb'] = (result['lgb1']+result['lgb2']+result['lgb3']+result['lgb4']+result['lgb5'])/5
result.to_csv('result/five_fold_all_day.csv',index=False)
print(result.describe())
col = ['lgb','lgb1', 'lgb2', 'lgb3', 'lgb4', 'lgb5']
for co in col:
    sub = result[['id',co]]
    sub.columns = ['id','score']
    sub.to_csv('result/five_fold_'+co+'_day.csv',index=False)
    print(co)
print(score)
print(np.array(score).mean())
print('Over')

(990006, 3557)
(500538, 3557)
[1]	train's auc: 0.970019	evals's auc: 0.967065
Training until validation scores don't improve for 50 rounds.
[2]	train's auc: 0.976418	evals's auc: 0.97486
[3]	train's auc: 0.980106	evals's auc: 0.978002
[4]	train's auc: 0.980965	evals's auc: 0.979257
[5]	train's auc: 0.981635	evals's auc: 0.979808
[6]	train's auc: 0.981938	evals's auc: 0.980249
[7]	train's auc: 0.982377	evals's auc: 0.980449
[8]	train's auc: 0.983403	evals's auc: 0.981617
[9]	train's auc: 0.983778	evals's auc: 0.981939
[10]	train's auc: 0.984631	evals's auc: 0.982296
[11]	train's auc: 0.984815	evals's auc: 0.982469
[12]	train's auc: 0.985215	evals's auc: 0.982629
[13]	train's auc: 0.985463	evals's auc: 0.982824
[14]	train's auc: 0.985615	evals's auc: 0.982988
[15]	train's auc: 0.986094	evals's auc: 0.983216
[16]	train's auc: 0.986387	evals's auc: 0.983461
[17]	train's auc: 0.986516	evals's auc: 0.983515
[18]	train's auc: 0.98672	evals's auc: 0.983663
[19]	train's auc: 0.986832	evals's au

[166]	train's auc: 0.992346	evals's auc: 0.98813
[167]	train's auc: 0.992362	evals's auc: 0.988106
[168]	train's auc: 0.992409	evals's auc: 0.98822
[169]	train's auc: 0.992457	evals's auc: 0.9883
[170]	train's auc: 0.992459	evals's auc: 0.988305
[171]	train's auc: 0.992483	evals's auc: 0.988329
[172]	train's auc: 0.992477	evals's auc: 0.988323
[173]	train's auc: 0.992455	evals's auc: 0.988315
[174]	train's auc: 0.992474	evals's auc: 0.988325
[175]	train's auc: 0.992502	evals's auc: 0.988331
[176]	train's auc: 0.992495	evals's auc: 0.98833
[177]	train's auc: 0.992492	evals's auc: 0.988328
[178]	train's auc: 0.992527	evals's auc: 0.988336
[179]	train's auc: 0.992546	evals's auc: 0.988329
[180]	train's auc: 0.992555	evals's auc: 0.988335
[181]	train's auc: 0.992581	evals's auc: 0.988346
[182]	train's auc: 0.992573	evals's auc: 0.988339
[183]	train's auc: 0.99257	evals's auc: 0.988343
[184]	train's auc: 0.992562	evals's auc: 0.988339
[185]	train's auc: 0.992579	evals's auc: 0.988348
[186]	

[331]	train's auc: 0.993935	evals's auc: 0.989425
[332]	train's auc: 0.993931	evals's auc: 0.989422
[333]	train's auc: 0.993952	evals's auc: 0.98944
[334]	train's auc: 0.99398	evals's auc: 0.989461
[335]	train's auc: 0.994001	evals's auc: 0.98951
[336]	train's auc: 0.994029	evals's auc: 0.989511
[337]	train's auc: 0.994024	evals's auc: 0.989508
[338]	train's auc: 0.994022	evals's auc: 0.989505
[339]	train's auc: 0.994018	evals's auc: 0.989501
[340]	train's auc: 0.994045	evals's auc: 0.989528
[341]	train's auc: 0.994041	evals's auc: 0.989527
[342]	train's auc: 0.994056	evals's auc: 0.989546
[343]	train's auc: 0.994072	evals's auc: 0.989568
[344]	train's auc: 0.9941	evals's auc: 0.989596
[345]	train's auc: 0.994126	evals's auc: 0.989627
[346]	train's auc: 0.994121	evals's auc: 0.989624
[347]	train's auc: 0.994147	evals's auc: 0.98964
[348]	train's auc: 0.994179	evals's auc: 0.989651
[349]	train's auc: 0.994209	evals's auc: 0.989686
[350]	train's auc: 0.994261	evals's auc: 0.989699
[351]	

[496]	train's auc: 0.995778	evals's auc: 0.990616
[497]	train's auc: 0.995799	evals's auc: 0.990624
[498]	train's auc: 0.995823	evals's auc: 0.990664
[499]	train's auc: 0.995851	evals's auc: 0.990655
[500]	train's auc: 0.995846	evals's auc: 0.990653
[501]	train's auc: 0.995843	evals's auc: 0.990649
[502]	train's auc: 0.99587	evals's auc: 0.990668
[503]	train's auc: 0.995885	evals's auc: 0.990666
[504]	train's auc: 0.995907	evals's auc: 0.990695
[505]	train's auc: 0.995906	evals's auc: 0.990693
[506]	train's auc: 0.995906	evals's auc: 0.990694
[507]	train's auc: 0.99592	evals's auc: 0.990689
[508]	train's auc: 0.995916	evals's auc: 0.990689
[509]	train's auc: 0.995934	evals's auc: 0.9907
[510]	train's auc: 0.995931	evals's auc: 0.990698
[511]	train's auc: 0.995927	evals's auc: 0.990695
[512]	train's auc: 0.995922	evals's auc: 0.990693
[513]	train's auc: 0.995933	evals's auc: 0.99069
[514]	train's auc: 0.995929	evals's auc: 0.990688
[515]	train's auc: 0.995951	evals's auc: 0.990668
[516]

[661]	train's auc: 0.996787	evals's auc: 0.991156
[662]	train's auc: 0.996788	evals's auc: 0.991156
[663]	train's auc: 0.996795	evals's auc: 0.991152
[664]	train's auc: 0.996801	evals's auc: 0.991166
[665]	train's auc: 0.996797	evals's auc: 0.991165
[666]	train's auc: 0.996796	evals's auc: 0.991165
[667]	train's auc: 0.996795	evals's auc: 0.991163
[668]	train's auc: 0.996793	evals's auc: 0.991162
[669]	train's auc: 0.996793	evals's auc: 0.991163
[670]	train's auc: 0.996793	evals's auc: 0.991164
[671]	train's auc: 0.996795	evals's auc: 0.991174
[672]	train's auc: 0.996795	evals's auc: 0.991174
[673]	train's auc: 0.996803	evals's auc: 0.99117
[674]	train's auc: 0.996808	evals's auc: 0.991185
[675]	train's auc: 0.996814	evals's auc: 0.99118
[676]	train's auc: 0.996821	evals's auc: 0.991193
[677]	train's auc: 0.996818	evals's auc: 0.991191
[678]	train's auc: 0.996815	evals's auc: 0.99119
[679]	train's auc: 0.996823	evals's auc: 0.991186
[680]	train's auc: 0.996832	evals's auc: 0.991179
[68

...

[752]	train's auc: 0.997062	evals's auc: 0.99239
[753]	train's auc: 0.99706	evals's auc: 0.992389
[754]	train's auc: 0.997068	evals's auc: 0.992399
[755]	train's auc: 0.997067	evals's auc: 0.992397
[756]	train's auc: 0.997068	evals's auc: 0.992398
[757]	train's auc: 0.997071	evals's auc: 0.992393
[758]	train's auc: 0.997079	evals's auc: 0.99239
[759]	train's auc: 0.997079	evals's auc: 0.99239
[760]	train's auc: 0.997075	evals's auc: 0.992389
[761]	train's auc: 0.997074	evals's auc: 0.992388
[762]	train's auc: 0.997073	evals's auc: 0.992388
[763]	train's auc: 0.997077	evals's auc: 0.99239
[764]	train's auc: 0.997077	evals's auc: 0.99239
[765]	train's auc: 0.997086	evals's auc: 0.992398
[766]	train's auc: 0.997094	evals's auc: 0.9924
[767]	train's auc: 0.997093	evals's auc: 0.992399
[768]	train's auc: 0.997094	evals's auc: 0.992399
[769]	train's auc: 0.997103	evals's auc: 0.992402
[770]	train's auc: 0.997103	evals's auc: 0.992402
[771]	train's auc: 0.997106	evals's auc: 0.992402
[772]	tr

[917]	train's auc: 0.997468	evals's auc: 0.992543
[918]	train's auc: 0.997476	evals's auc: 0.992541
[919]	train's auc: 0.997476	evals's auc: 0.99254
[920]	train's auc: 0.997476	evals's auc: 0.992541
[921]	train's auc: 0.997481	evals's auc: 0.99255
[922]	train's auc: 0.997486	evals's auc: 0.992548
[923]	train's auc: 0.997485	evals's auc: 0.992547
[924]	train's auc: 0.997485	evals's auc: 0.992546
[925]	train's auc: 0.997491	evals's auc: 0.992551
[926]	train's auc: 0.99749	evals's auc: 0.99255
[927]	train's auc: 0.997488	evals's auc: 0.992549
[928]	train's auc: 0.997498	evals's auc: 0.992546
[929]	train's auc: 0.997503	evals's auc: 0.99255
[930]	train's auc: 0.997509	evals's auc: 0.992552
[931]	train's auc: 0.997508	evals's auc: 0.992552
[932]	train's auc: 0.997508	evals's auc: 0.992552
[933]	train's auc: 0.997516	evals's auc: 0.992538
[934]	train's auc: 0.997514	evals's auc: 0.992537
[935]	train's auc: 0.997521	evals's auc: 0.992534
[936]	train's auc: 0.997531	evals's auc: 0.992539
[937]

[1080]	train's auc: 0.998081	evals's auc: 0.992699
[1081]	train's auc: 0.998087	evals's auc: 0.99271
[1082]	train's auc: 0.998087	evals's auc: 0.99271
[1083]	train's auc: 0.998093	evals's auc: 0.992713
[1084]	train's auc: 0.998093	evals's auc: 0.992713
[1085]	train's auc: 0.998097	evals's auc: 0.992714
[1086]	train's auc: 0.998105	evals's auc: 0.992713
[1087]	train's auc: 0.998106	evals's auc: 0.992713
[1088]	train's auc: 0.998105	evals's auc: 0.992712
[1089]	train's auc: 0.998104	evals's auc: 0.992712
[1090]	train's auc: 0.998104	evals's auc: 0.992712
[1091]	train's auc: 0.99811	evals's auc: 0.992713
[1092]	train's auc: 0.998115	evals's auc: 0.992718
[1093]	train's auc: 0.998115	evals's auc: 0.99272
[1094]	train's auc: 0.99812	evals's auc: 0.992723
[1095]	train's auc: 0.99812	evals's auc: 0.992723
[1096]	train's auc: 0.99812	evals's auc: 0.992723
[1097]	train's auc: 0.998124	evals's auc: 0.992726
[1098]	train's auc: 0.998131	evals's auc: 0.99273
[1099]	train's auc: 0.998134	evals's au

[1242]	train's auc: 0.998533	evals's auc: 0.992862
[1243]	train's auc: 0.998536	evals's auc: 0.992868
[1244]	train's auc: 0.998539	evals's auc: 0.992869
[1245]	train's auc: 0.998539	evals's auc: 0.992868
[1246]	train's auc: 0.99854	evals's auc: 0.992869
[1247]	train's auc: 0.998544	evals's auc: 0.992861
[1248]	train's auc: 0.998545	evals's auc: 0.99286
[1249]	train's auc: 0.998545	evals's auc: 0.99286
[1250]	train's auc: 0.998545	evals's auc: 0.99286
[1251]	train's auc: 0.99855	evals's auc: 0.992861
[1252]	train's auc: 0.99855	evals's auc: 0.992861
[1253]	train's auc: 0.998554	evals's auc: 0.992864
[1254]	train's auc: 0.998558	evals's auc: 0.99286
[1255]	train's auc: 0.998559	evals's auc: 0.99286
[1256]	train's auc: 0.998559	evals's auc: 0.99286
[1257]	train's auc: 0.99856	evals's auc: 0.99286
[1258]	train's auc: 0.998562	evals's auc: 0.992865
[1259]	train's auc: 0.998565	evals's auc: 0.992866
[1260]	train's auc: 0.998565	evals's auc: 0.992866
[1261]	train's auc: 0.998566	evals's auc: 

[1404]	train's auc: 0.99875	evals's auc: 0.992905
[1405]	train's auc: 0.99875	evals's auc: 0.992905
[1406]	train's auc: 0.99875	evals's auc: 0.992905
[1407]	train's auc: 0.998751	evals's auc: 0.992905
[1408]	train's auc: 0.998751	evals's auc: 0.992906
[1409]	train's auc: 0.998754	evals's auc: 0.9929
[1410]	train's auc: 0.998754	evals's auc: 0.9929
[1411]	train's auc: 0.998755	evals's auc: 0.992901
[1412]	train's auc: 0.998758	evals's auc: 0.992904
[1413]	train's auc: 0.998758	evals's auc: 0.992903
[1414]	train's auc: 0.998758	evals's auc: 0.992904
[1415]	train's auc: 0.998759	evals's auc: 0.992905
[1416]	train's auc: 0.998762	evals's auc: 0.992902
[1417]	train's auc: 0.998765	evals's auc: 0.992897
[1418]	train's auc: 0.998767	evals's auc: 0.992898
[1419]	train's auc: 0.998769	evals's auc: 0.992898
[1420]	train's auc: 0.998771	evals's auc: 0.992896
[1421]	train's auc: 0.998771	evals's auc: 0.992895
[1422]	train's auc: 0.998771	evals's auc: 0.992895
[1423]	train's auc: 0.998771	evals's a